# 審査区分データの取得

ダウンロードしたXMLファイルから、課題番号ごとの審査区分データ（review_section）を取得して、ローカルのデータベースに保存します。

In [ ]:
from lxml import etree
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import pickle
import glob
import sqlalchemy as sa
import sqlalchemy.ext.declarative
import pymysql
import re

## 研究課題ごとの審査区分を抽出する関数

In [ ]:
def review_section(xmlfile):
    tree = etree.parse(xmlfile)
    nsmap = {"xml": "http://www.w3.org/XML/1998/namespace"}
    review_secitonlist = []
    for grantAward in tree.iterfind("grantAward"):
        awardnumber = grantAward.get("awardNumber")
        if grantAward.find("summary[@xml:lang='ja']/review_section", nsmap) is None:
            pass
        else:
            for review_section in grantAward.find("summary[@xml:lang='ja']", nsmap).iterfind("review_section", nsmap):
                review_section_sequence = review_section.get("sequence")
                review_section_niicode = review_section.get("niiCode")
                review_section_table_type = review_section.get("tableType")

                row = [
                    awardnumber,
                    review_section_sequence,
                    review_section_niicode,
                    review_section_table_type,
                ]
                review_secitonlist.append(row)
            
    df = pd.DataFrame(review_secitonlist)
    
    if len(df.index) > 0:
        df.columns = [
            'awardnumber',
            'review_section_sequence',
            'review_section_niicode',
            'review_section_table_type',
        ]
    
        pickledfile = 'pickledDF_review_section/' + re.search('[0-9]{4}_[0-9]+-[0-9]+.xml', xmlfile).group() + '.dump'
        df.to_pickle(pickledfile)

xmlファイルをパースして、pickleして保存

In [ ]:
import os
import shutil

def cleandir(dirname):
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

cleandir('pickledDF_review_section')

for i in tqdm(range(2018, 2019)):
    globdir = 'xml/' + str(i) + '*.xml'
    for xmlfile in glob.glob(globdir):
        review_section(xmlfile)

空のデータフレームを作って…

In [ ]:
columns = [
    'awardnumber',
    'review_section_sequence',
    'review_section_niicode',
    'review_section_table_type',
]
df = pd.DataFrame(columns=columns)

pickleからデータフレームを復元、すべて結合

In [ ]:
for dump in tqdm(glob.glob('pickledDF_review_section/*.dump')):
    with open(dump, mode='rb') as f:
        df = pd.concat([df, pickle.load(f)])

連結したデータフレームをいったんpickleして保存

In [ ]:
df.to_pickle('beforeCleaning/parse_review_section.dump')

### データクリーニング

pickleからデータフレームを復元する

In [ ]:
with open('beforeCleaning/parse_review_section.dump', mode='rb') as f:
    df = pickle.load(f)

In [ ]:
df.info()

データ型を指定する

In [ ]:
df = df.fillna(0)
df = df.astype({
    'review_section_sequence': np.int64,
    'review_section_niicode': np.int64,
})
df.info()

awardnumberをインデックスに設定する

In [ ]:
df = df.set_index('awardnumber')

ローカルのMariaDBに保存する

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('../settings/config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer

engine = create_engine(url, echo=True)

df.to_sql('grantaward_review_section', engine, if_exists='replace',
      dtype={
          'awardnumber': String(256),
          'review_section_sequence': Integer,
          'review_section_niicode': Integer,
          'review_section_table': String(256),
      })

## おしまい

データがコミットされていれば終了。HeidiSQLなどで、上記データフレームの件数が登録されているか確認する。